In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, t
import seaborn as sns
from scipy.optimize import minimize

Get map data created in '1_combining_datasets.ipynb'

In [ ]:
map_df = gpd.read_file("../midsave/map_crime.gpkg")

Define parameters of interest

In [ ]:
cities = ['Bordeaux', 'Clermont-Ferrand', 'Dijon', 'Grenoble', 'Lille',
                 'Lyon', 'Mans', 'Marseille', 'Metz', 'Montpellier',
                 'Nancy', 'Nantes', 'Nice', 'Orleans', 'Paris',
                 'Rennes', 'Saint-Etienne', 'Strasbourg', 'Toulouse', 'Tours']

apps = ['Web_Adult', 'Tor', 'YouTube']

traffic_dir = ['DL']

Extract commune-level time series

In [ ]:
df = pd.DataFrame()

for city_str in tqdm(cities):
    
    map_city = map_df[map_df.cities == city_str]
    
    df_0 = pd.DataFrame()
    
    for app_str in apps:
        for rate_str in traffic_dir:
            
            df_1 = pd.DataFrame()
            
            for month in range(3, 6):
              traffic = dict()
              s = 1
              if month == 3:
                s = 16
              if month == 4:
                n = 31
              else:
                n = 32
              for day in range(s, n):
                day_index = day
                if day < 10:
                  day_str = f'20190{month}0{day}'
                else:
                  day_str = f'20190{month}{day}'

                day_print = datetime.datetime.strptime(day_str, '%Y%m%d')
                times = [day_print + datetime.timedelta(minutes=15*i) for i in range(96)]
                times_str = [t.strftime('%H:%M') for t in times]

                # column names
                columns = ['tile_id'] + times_str
                
                df_2 = pd.read_csv(f'../Data/Netmob/{city_str}/{app_str}/{day_str}/{city_str}_{app_str}_{day_str}_{rate_str}.txt', sep = " ", names = columns)
                
                df_2_list= list(df_2)
                df_2_list.remove('tile_id')
                
                df_2 = (df_2.merge(map_city, on = 'tile_id', how = 'left')
                        .groupby(['code_dep', 'code_com'])[df_2_list]
                        .sum()
                        .reset_index()
                        .copy())
                
                df_2["date"] = day_str
                
                df_1 = pd.concat([df_1,df_2])
            
            df_1 = pd.melt(df_1, id_vars=['code_dep', 'code_com', 'date'], var_name='time', value_name=app_str).copy()
                                  
            if df_0.empty:
                df_0 = df_1.copy()
            else:
                df_0 = df_0.merge(df_1, on = ['code_dep', 'code_com', 'date', 'time'], how = 'left').reset_index().copy()
            
            df_0["traffic_dir"] = rate_str
            df_0["cities"] = city_str            
            
    df = pd.concat([df,df_0])

In [ ]:
df['hour'] = df['time'].str.split(':').str[0].astype(int)

In [ ]:
df.drop_duplicates(subset = ['code_com', 'date', 'time'], inplace = True)

In [ ]:
df.shape

In [ ]:
df.head()

Share of time windows across communes without any Tor traffic

In [ ]:
(df['Tor'] == 0).sum()/df.shape[0]

### Filter out network downtimes

According to Netmob documentation, some network anomalies due to service interruptions on March 31 and May 12 occurred. Not to distort downstream analysis, we filter out those anomalies -- the large one by date and smaller ones by with traffic to porn sites being zero.

In [ ]:
df = df[~((df['Web_Adult'] == 0) & ((df['date'] == '20190512') | (df['date'] == '20190331')))]

In [ ]:
df.shape

# Calculate child pornography consumption

In [ ]:
cp_com = (df
             .drop(columns=['time', 'date', 'hour'])
             .groupby(['cities', 'code_com', 'traffic_dir'])
             .sum()
             .reset_index()
             .copy())

In [ ]:
cp_com.head()

In [ ]:
cp_com.shape

### Calculate correction factor by correlation time series by commune

We define four different correction factors that we then compare against the correlation with sexual violence rates:

1) c1 = Pearsons rho
2) c2 = Persons rho**2 
3) c3 = (Pearsons rho - mean(Pearsons rho) + 1)*c_global
4) Spearmans rho


In [ ]:
df_hour = (df
             .drop(columns=['time'])
             .groupby(['date', 'hour', 'code_com', 'cities', 'traffic_dir'])
             .sum()
             .reset_index()
             .copy())

In [ ]:
def log_values(x):
    return np.log(x)

In [ ]:
df_hour.shape

In [ ]:
df_hour.head()

To avoid running into log problems, we replace zeros with tiny numbers.

In [ ]:
df_hour['date'] = pd.to_numeric(df_hour['date'])
df_hour.loc[df_hour.Tor == 0, 'Tor'] = 0.001
df_hour.loc[df_hour.Web_Adult == 0, 'Web_Adult'] = 0.001
df_hour.loc[df_hour.YouTube == 0, 'YouTube'] = 0.001

In [ ]:
df_hour['Web_Adult_log'] = df_hour['Web_Adult'].transform(log_values)
df_hour['YouTube_log'] = df_hour['YouTube'].transform(log_values)
df_hour['Tor_log'] = df_hour['Tor'].transform(log_values)

Filter out network downtimes

In [ ]:
df_hour.shape

In [ ]:
def correlation_coefficient_log_pear(x):
    return x['Web_Adult_log'].corr(x['Tor_log'], method='pearson')
def correlation_coefficient_log_spear(x):
    return x['Web_Adult_log'].corr(x['Tor_log'], method='spearman')

In [ ]:
correction_factor = (pd.DataFrame(df_hour
                  .groupby('code_com')
                  .apply(correlation_coefficient_log_pear), columns=['c']
                  )
      .reset_index())

The correlation coefficient captures Tor correlation with any type of porn consumption. Thus, we multiply by the correction factor derived from the DUTA dataset -> 105/253 and the share of Tor traffic that goes to onion websites -> 0.011

In [ ]:
correction_factor['c'] = correction_factor['c']*(105/253)*0.011

How many correlation coefficients are smaller than 0?

In [ ]:
correction_factor.loc[correction_factor.c < 0, 'c'].size

In [ ]:
correction_factor.loc[correction_factor.c < 0, 'c'] = 0.00001

Merge to main table

In [ ]:
cp_com = cp_com.merge(correction_factor, on = 'code_com', how = 'left')

### Add population data

In [ ]:
cp_com = cp_com.merge(map_df[['code_com', 'pop']].drop_duplicates('code_com'), on = 'code_com', how = 'left')

In [ ]:
cp_com.head()

### Assign global factor representing the share of Tor traffic to darknet websites that are linked to pornographic content

- Source of tor_to_darknet_share: 
    - Lower bound: 0.011 https://metrics.torproject.org/bandwidth.html?start=2017-01-01&end=2019-12-31
    - Middle bound: 0.034 https://blog.torproject.org/some-statistics-about-onions/
    - Upper bound: 0.078 https://www.pnas.org/doi/10.1073/pnas.2011893117#
- Source of Pornography_in_darknet_share: https://arxiv.org/pdf/2305.08596.pdf (Table 9)
- Source of Child_pornography_to_pornography_in_darknet_share: DUTA dataset (share of sub-category child porn in category porn)

In [ ]:
cp_com['Tor_to_darknet_share'] = 0.011 #0.023 #0.977 or alternatively 0.078 from PNAS paper
cp_com['Pornography_in_darknet_URL_share'] = 2267628/5437248
cp_com['Child_pornography_to_pornography_in_darknet_URL_share'] = 105/253

In [ ]:
pd.unique(cp_com['Tor_to_darknet_share'] * cp_com['Pornography_in_darknet_URL_share'] * cp_com['Child_pornography_to_pornography_in_darknet_URL_share'])*100

### Calculate consumption estimate

In [ ]:
cp_com['cpc'] = cp_com['Tor'] * cp_com['c']
cp_com['cpc_per_1000'] = cp_com['Tor'] * cp_com['c'] * 1000 / cp_com['pop']

# Same in log
cp_com['log_cpc'] = np.log(cp_com['Tor'] * cp_com['c'])
cp_com['log_cpc_per_1000'] = np.log(cp_com['Tor'] * cp_com['c']) * 1000 / cp_com['pop']    

Get other web service traffic also per 1000

In [ ]:
cp_com['yt_per_1000'] = cp_com['YouTube'] / cp_com['pop'] * 1000
cp_com['wa_per_1000'] = cp_com['Web_Adult'] / cp_com['pop'] * 1000
cp_com['tor_per_1000'] = cp_com['Tor'] / cp_com['pop'] * 1000

In [ ]:
cp_com['log_yt_per_1000'] = np.log(cp_com['YouTube']) / cp_com['pop'] * 1000
cp_com['log_wa_per_1000'] = np.log(cp_com['Web_Adult']) / cp_com['pop'] * 1000
cp_com['log_tor_per_1000'] = np.log(cp_com['Tor']) / cp_com['pop'] * 1000

In [ ]:
cp_com.head()

In [ ]:
cp_com.shape

### Testing for correlations with commune-level sexual violence rates

https://www.data.gouv.fr/fr/datasets/bases-statistiques-communale-et-departementale-de-la-delinquance-enregistree-par-la-police-et-la-gendarmerie-nationales/

They provide data for some communes directly and for those that have less than 5 cases reported in 3 consecutive years, they provide the department-level average per year.

In [ ]:
crime = pd.read_csv('../external_data/donnee-data.gouv-2022-geographie2023-produit-le2023-07-17.csv', delimiter=';', decimal=",", dtype={'CODGEO_2023': str, 'tauxpourmille': float, 'complementinfotaux': float})

In [ ]:
crime = (crime
    .query("classe == 'Violences sexuelles'")
    .query("annee == [17, 18, 19, 20, 21]")
    .rename(columns={'CODGEO_2023':'code_com',
                     'tauxpourmille':'sv_per_1000_com',
                     'complementinfotaux':'sv_per_1000_est',
                    'annee':'date',
                    'classe':'indicator'
                    })[['code_com', 'sv_per_1000_com', 'sv_per_1000_est', 'date', 'indicator']]
)

In [ ]:
crime['sv_per_1000'] = crime['sv_per_1000_com'].fillna(crime['sv_per_1000_est'])

In [ ]:
crime.head()

Calculate rate for 2019 and a 5-year average

In [ ]:
df_crime = (crime
 .query("date == 19")
 .groupby(['code_com'])['sv_per_1000'].mean()
 .reset_index()
 .rename(columns = {'sv_per_1000':'sv_19'})
 .merge(crime.query("date == [17, 18, 19, 20, 21]")
        .groupby(['code_com'])['sv_per_1000'].mean()
        .reset_index()
        .rename(columns = {'sv_per_1000':'sv_17_21'}))
 .merge(crime.query("date == [19]")
        .groupby(['code_com'])['sv_per_1000_com'].mean()
        .reset_index()
        .rename(columns = {'sv_per_1000_com':'sv_com_19'}))
 .merge(crime.query("date == [17, 18, 19, 20, 21]")
        .groupby(['code_com'])['sv_per_1000_com'].mean()
        .reset_index()
        .rename(columns = {'sv_per_1000_com':'sv_com_17_21'})))

In [ ]:
df_crime.head()

In [ ]:
crime.head()

Merge with cpc estimates

In [ ]:
cp_com = cp_com.merge(df_crime, on = 'code_com', how = 'left').drop_duplicates(['code_com'])

In [ ]:
cp_com.head()

In [ ]:
cp_com.shape

In [ ]:
cp_com.to_csv("../midsave/cpc_com.csv", index=False)

### Getting Spearman's rho and test for different from null

In [ ]:
test = (cp_com[['yt_per_1000', 'wa_per_1000', 'tor_per_1000', 
                'cpc_per_1000', 'sv_17_21', 'sv_com_17_21']]
        .dropna(subset=['sv_com_17_21']))

In [ ]:
test['yt_per_1000'] = np.log(test['yt_per_1000'])
test['wa_per_1000'] = np.log(test['wa_per_1000'])
test['tor_per_1000'] = np.log(test['tor_per_1000'])
test['cpc_per_1000'] = np.log(test['cpc_per_1000'])

In [ ]:
test.shape

Get p-values

In [ ]:
spearmanr(test['yt_per_1000'], test['sv_com_17_21'])

In [ ]:
spearmanr(test['wa_per_1000'], test['sv_com_17_21'])

In [ ]:
spearmanr(test['tor_per_1000'], test['sv_com_17_21'])

In [ ]:
spearmanr(test['cpc_per_1000'], test['sv_com_17_21'])

In [ ]:
spearmanr(test['yt_per_1000'], test['cpc_per_1000'])

In [ ]:
def dependent_corr_test(r_ab, r_ac, r_bc, n):
    """
    Perform a paired-samples test for dependent correlation coefficients.
    
    Arguments:
    r_ab -- correlation coefficient between variable A and B
    r_ac -- correlation coefficient between variable A and C
    r_bc -- correlation coefficient between variable B and C
    n -- sample size
    
    Returns:
    t_value -- t-value
    p_value -- p-value
    """
    
    r_diff = r_ab - r_ac
    se_diff = np.sqrt((1 - r_bc**2) / (n - 3))
    
    t_value = r_diff / se_diff
    p_value = 2 * (1 - t.cdf(abs(t_value), df=n-2))
    
    return t_value, p_value

CPC-YT

In [ ]:
r_ab = spearmanr(test['yt_per_1000'], test['sv_com_17_21'])[0]
r_ac = spearmanr(test['cpc_per_1000'], test['sv_com_17_21'])[0]
r_bc = spearmanr(test['yt_per_1000'], test['cpc_per_1000'])[0]
n = 731

t_value, p_value = dependent_corr_test(r_ab, r_ac, r_bc, n)
print("t-value:", t_value)
print("p-value:", p_value)

CPC-WA

In [ ]:
r_ab = spearmanr(test['wa_per_1000'], test['sv_com_17_21'])[0]
r_ac = spearmanr(test['cpc_per_1000'], test['sv_com_17_21'])[0]
r_bc = spearmanr(test['cpc_per_1000'], test['wa_per_1000'])[0]
n = 731

t_value, p_value = dependent_corr_test(r_ab, r_ac, r_bc, n)
print("t-value:", t_value)
print("p-value:", p_value)

CPC-Tor

In [ ]:
r_ab = spearmanr(test['tor_per_1000'], test['sv_com_17_21'])[0]
r_ac = spearmanr(test['cpc_per_1000'], test['sv_com_17_21'])[0]
r_bc = spearmanr(test['cpc_per_1000'], test['tor_per_1000'])[0]
n = 731

t_value, p_value = dependent_corr_test(r_ab, r_ac, r_bc, n)
print("t-value:", t_value)
print("p-value:", p_value)

In [ ]:
dependent_corr_test(r_ab, r_ac, r_bc, n)

# Generating Heatmap data

In [ ]:
df_hour = df_hour.merge(cp_com[['code_com', 'c']], on = ['code_com'], how = 'left')

In [ ]:
df_hour['Tor_scaled'] = df_hour['Tor'] * df_hour['c']

#### For all areas

In [ ]:
df_hour.head(1)

In [ ]:
df_hour['date'] = pd.to_datetime(df_hour['date'], format='%Y%m%d')

In [ ]:
df_hour['day'] = df_hour['date'].dt.strftime('%A')

In [ ]:
df_heat = (df_hour
           .rename(columns={'hour': 'Hour', 'day': 'Day'})
           .groupby(['Day', 'Hour'])[['Tor_scaled','Web_Adult', 'YouTube']]
                        .sum()
                        .reset_index())

In [ ]:
df_heat.to_csv("../midsave/heatmap.csv", index=False)

#### For top 10 highest cpc areas

In [ ]:
cp_com.sort_values(by='cpc_per_1000', ascending=False)

In [ ]:
cp_com.sort_values(by='cpc_per_1000', ascending=False, inplace=True)

In [ ]:
df_heat_10 = (df_hour
           .query('code_com in @cp_com.code_com.head(10)')
           .rename(columns={'hour': 'Hour', 'day': 'Day'})
           .groupby(['Day', 'Hour'])[['Tor_scaled','Web_Adult', 'YouTube']]
                        .sum()
                        .reset_index())
df_heat_10.to_csv("../midsave/heatmap_10.csv", index=False)

#### Most CPC_per_1000 area

 Number 1

In [ ]:
cp_com['code_com'].iloc[0:1].tolist()

In [ ]:
df_heat_1 = df_hour[df_hour['code_com'].isin(cp_com['code_com'].iloc[0:1].tolist())]

df_heat_1 = (df_heat_1
             .rename(columns={'hour': 'Hour', 'day': 'Day'})
             .groupby(['Day', 'Hour'])[['Tor_scaled','Web_Adult', 'YouTube']]
             .sum()
             .reset_index())
df_heat_1.to_csv("../midsave/heatmap_1_31352.csv", index=False)

Number 2

In [ ]:
cp_com['code_com'].iloc[1:2].tolist()

In [ ]:
df_heat_2 = df_hour[df_hour['code_com'].isin(cp_com['code_com'].iloc[1:2].tolist())]

df_heat_2 = (df_heat_2
             .rename(columns={'hour': 'Hour', 'day': 'Day'})
             .groupby(['Day', 'Hour'])[['Tor_scaled','Web_Adult', 'YouTube']]
             .sum()
             .reset_index())
df_heat_2.to_csv("../midsave/heatmap_2_21192.csv", index=False)

Number 3

In [ ]:
cp_com['code_com'].iloc[2:3].tolist()

In [ ]:
df_heat_3 = df_hour[df_hour['code_com'].isin(cp_com['code_com'].iloc[2:3].tolist())]

df_heat_3 = (df_heat_3
             .rename(columns={'hour': 'Hour', 'day': 'Day'})
             .groupby(['Day', 'Hour'])[['Tor_scaled','Web_Adult', 'YouTube']]
             .sum()
             .reset_index())
df_heat_3.to_csv("../midsave/heatmap_3_45072.csv", index=False)